<a href="https://colab.research.google.com/github/Tower5954/TensorFlow/blob/main/05_Transfer_learning_in_TensorFlow_part_2_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer learning with TensorFlow part 2: Fine-tuning

In the previous notebook, we covered transfer learning feature extraction, in this notebook we will go through a new kind of transfer learning: fine-tuning

In [1]:
# Check if we are using a GPU
!nvidia-smi

Mon Dec 20 05:35:25 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Creating helper functions

It is a good idea, to put functions that you will use a lot in a script, that you can download and import into your notebooks (or elsewhere).

We have done this for some of the functions previously used here:
https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2021-12-20 05:35:25--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-12-20 05:35:25 (28.8 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
from helper_functions import create_tensorboard_callback,plot_loss_curves,unzip_data,walk_through_dir

## The data

This time we will see how we can use the pretrained models within tf.keras.applications and apply them to our own problem (recognising images of food)
link: https://www.tensorflow.org/api_docs/python/tf/keras/applications

In [4]:
# Get 10% of training data of 10 classes of food 101

In [5]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

--2021-12-20 05:35:31--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 209.85.145.128, 172.217.219.128, 209.85.146.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|209.85.145.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M   183MB/s    in 0.9s    

2021-12-20 05:35:32 (183 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [6]:
unzip_data("10_food_classes_10_percent.zip")

In [7]:
# check how many images and subdirectories in the dataset

In [8]:
walk_through_dir("10_food_classes_10_percent")

There are 2 directories and 0 images in '10_food_classes_10_percent'.
There are 10 directories and 0 images in '10_food_classes_10_percent/test'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/fried_rice'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/grilled_salmon'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/steak'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/hamburger'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/chicken_curry'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/pizza'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/sushi'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/ramen'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/chicken_wings'.
There are 0 directories and 250 images in '10_food_classes_10_percent/tes

In [9]:
# Create training and test 

In [10]:
train_dir = "10_food_classes_10_percent/train" 
test_dir = "10_food_classes_10_percent/test"

In [11]:
import tensorflow as tf
IMG_SIZE = (224,224)
BATCH_SIZE = 32

train_data_10_percent = tf.keras.preprocessing.image_dataset_from_directory(directory=train_dir,
                                                                            image_size=IMG_SIZE,
                                                                            label_mode="categorical",
                                                                            batch_size=BATCH_SIZE)


test_data = tf.keras.preprocessing.image_dataset_from_directory(directory=test_dir,
                                                                image_size=IMG_SIZE,
                                                                label_mode="categorical",
                                                                batch_size=BATCH_SIZE)

Found 750 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


In [12]:
train_data_10_percent

<BatchDataset shapes: ((None, 224, 224, 3), (None, 10)), types: (tf.float32, tf.float32)>

In [13]:
# Check the class names of the dataset

In [14]:
train_data_10_percent.class_names

['chicken_curry',
 'chicken_wings',
 'fried_rice',
 'grilled_salmon',
 'hamburger',
 'ice_cream',
 'pizza',
 'ramen',
 'steak',
 'sushi']

In [15]:
# See an example of a batch of data

for images, labels in train_data_10_percent.take(1):
  print(images,labels)

tf.Tensor(
[[[[1.00000000e+00 2.00000000e+00 0.00000000e+00]
   [1.00000000e+00 2.00000000e+00 0.00000000e+00]
   [3.00000000e+00 2.00000000e+00 0.00000000e+00]
   ...
   [0.00000000e+00 1.00000000e+00 3.00000000e+00]
   [0.00000000e+00 1.00000000e+00 3.00000000e+00]
   [0.00000000e+00 2.00000000e+00 3.00000000e+00]]

  [[1.00000000e+00 2.00000000e+00 0.00000000e+00]
   [2.06632662e+00 1.06632650e+00 0.00000000e+00]
   [3.00000000e+00 2.00000000e+00 0.00000000e+00]
   ...
   [0.00000000e+00 1.00000000e+00 3.00000000e+00]
   [0.00000000e+00 1.00000000e+00 3.00000000e+00]
   [0.00000000e+00 1.00000000e+00 3.00000000e+00]]

  [[2.29081655e+00 1.71938777e+00 0.00000000e+00]
   [2.98469400e+00 1.98469388e+00 0.00000000e+00]
   [3.00000000e+00 2.00000000e+00 0.00000000e+00]
   ...
   [0.00000000e+00 1.00000000e+00 3.00000000e+00]
   [0.00000000e+00 1.00000000e+00 3.00000000e+00]
   [0.00000000e+00 1.00000000e+00 3.00000000e+00]]

  ...

  [[8.21429443e+00 2.14294434e-01 0.00000000e+00]
   [8

## Model 0: Building a transfer learning feature extraction model using the keras functional API.

---



The sequential API is straightforward, it runs the layers in sequential order.

However, the functional API  gives more flexiability with models.
link: https://www.tensorflow.org/guide/keras/functional?hl=en

**1.** Create a base model with tf.keras.applications.

In [17]:
base_model = tf.keras.applications.EfficientNetB0(include_top=False)

16719872/16705208 [==============================] - 0s 0us/step




---



**2**. Freeze the base model (so the underlying pre-trained patterns aren't updated during training).

In [18]:
base_model.trainable = False



---






**3**. Create inputs in to the model.

In [19]:
inputs = tf.keras.layers.Input(shape=(254,254,3), name='input_layer')





---






**4**. If using ResNet50V2 you will need to normalise inputs (you don't have to for EffecientNet(s)).


In [20]:
# x = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)(inputs)



---



**5**. Pass the inputs to the base_model.

In [21]:
x = base_model(inputs)
print(f"Shape after passing inputs through base model: {x.shape} ")

Shape after passing inputs through base model: (None, 7, 7, 1280) 




---





**6**. Average pool the outputs of the base model (aggregate all the most importantinformation, reduce number of computations). 

In [22]:
x = tf.keras.layers.GlobalAveragePooling2D(name='global_avg_pooling_layer')(x)
print(f"Shape after GlobalAveragePooling2d: {x.shape}")

Shape after GlobalAveragePooling2d: (None, 1280)




---

**7**. Create the output activation layer.

In [23]:
outputs = tf.keras.layers.Dense(10, activation="softmax", name="output_layer")(x)



---

**8**. Combine the inputs with the outputs into a model.

In [24]:
model_0 = tf.keras.Model(inputs,outputs)



---

**9**. Compile for the model